In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [2]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [3]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [6]:
sweep_config = {

     'method': 'grid',

    'parameters': {
        'learning_rate': 
         {'values': [1e-2,1e-3,1e-4,]},
        
        'Drop_1': 
            {'values': [ 0.25,0.35,0.45,0.15]
                   
                   
                },
        
        'Drop_2': 
            {'values': [ 0.5,0.6,0.4]
                   
                   
                }
     
    
        }

    }
sweep_id = wandb.sweep(sweep_config, project= "MINI_VGG_4blocks_With_BN_DropOut") 

Create sweep with ID: zjyxsm43
Sweep URL: https://app.wandb.ai/roastedkernel/MINI_VGG_4blocks_With_BN_DropOut/sweeps/zjyxsm43


In [ ]:
def train():
    wandb.init()
    configs = {'learning_rate': 0.0001,'Drop_1': 0.25, 'Drop_2': 0.5 }
    config = wandb.config
    config.epochs = 100
    
    
    
    model6 = Sequential()
    model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model6.add(BatchNormalization())
    model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(MaxPooling2D((2, 2)))
    model6.add(Dropout(wandb.config.Drop_1))
    model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(MaxPooling2D((2, 2)))
    model6.add(Dropout(wandb.config.Drop_1))
    model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(MaxPooling2D((2, 2)))
    model6.add(Dropout(wandb.config.Drop_1))
    model6.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model6.add(BatchNormalization())
    model6.add(MaxPooling2D((2, 2)))
    model6.add(Dropout(wandb.config.Drop_1))
    model6.add(Flatten())
    model6.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model6.add(BatchNormalization())
    model6.add(Dropout(wandb.config.Drop_2))
    model6.add(Dense(10, activation='softmax'))
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=35)

    metric = "val_accuracy"

    checkpoint = ModelCheckpoint("MINI_VGG.h5", monitor= metric, verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto')
    
    
    #n_epochs = 100
    batch = 2048
    opt = Adam(lr=wandb.config.learning_rate,amsgrad=True)
    
    
    model6.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=['accuracy'])
    
    wandb_callback = WandbCallback(data_type= "image", validation_data=(testX, testY)
                                 , labels = labelNames)

    history = model6.fit(trainX, trainY, epochs=config.epochs, 
                    validation_data=(testX, testY), use_multiprocessing=True,
                   callbacks=[es,checkpoint,wandb_callback], batch_size=batch,verbose=10)


In [ ]:
wandb.agent(sweep_id, function=train)